<a href="https://colab.research.google.com/github/MJMortensonWarwick/AAMA/blob/main/1_3_i_feel_a_need_for_speed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speed it Up!
This tutorial will look at techniques for speeding up Python and algorithms.

Let's start with some _list comprehension_:

In [5]:
# slow way
square_numbers = []
for number in range(1, 10):
	 square_numbers.append(number ** 2)

# faster way
square_numbers = [number ** 2 for number in range(1, 10)]
square_numbers

[1, 4, 9, 16, 25, 36, 49, 64, 81]

Both these techniques produce the same output, but the second one is much faster and briefer (one line less of code).

Let's try another:

In [6]:
# slow way
sentence = "This " + "is " + "a " + "slower " + "way " + "to " + "make " + "a " + "sentence."
print(sentence)

# faster way
mytuple = ("This", "is", "a", "faster", "way", "to", "make", "a", "sentence.")
sentence = " ".join(mytuple)
print(sentence)

This is a slower way to make a sentence.
This is a faster way to make a sentence.


By converting from concatenation (example #1 - the "slow way") we are always going to be slower than joining (example #2). The syntax here is that we specify how to join (a space - " ") and then pass the join command and the tuple of text we want to join.

How about unions?



In [7]:
a = [1,1,2,3]
b = [2,3,4,5]

# slow way
union_nos = []
for x in a:
  for y in b:
    if x == y:
      if x not in union_nos:
        union_nos.append(x)

# faster way
union_nos  = set(a) & set(b)
union_nos

{2, 3}

So here we are asking Python to find all the unique elements in list _a_ that are also in _b_ ... i.e. the intersection of the two lists. In the first ("slow way") we have a for loop inside a loop (always a bad sign) that check the values in _a_ and then checks if they are in _b_. If they are, and are not already added, we added them to the list.

Version two uses _set_ to find the unique values in each and then uses ampersand ("&") to find those in the intersection. Again, far less code and far quicker.

How about sorting?

In [8]:
# quick sorting
import operator

a = [ (1,'z'), (2, 'x'), (3, 'y') ]
a.sort(key=operator. itemgetter(1))
a

[(2, 'x'), (3, 'y'), (1, 'z')]

Here we are using _itemgetter_ for quick sorting. We specify "(1)" to say we want to sort by the second item (remember, first item in Python is 0 and seond is 1). This is much faster than other forms of sorting.

## Vectorisation and Numba
To end the tutorial we will use compilation techniques to speed up our code. First we will use the built-in _numpy_ code to vectorise our Python and make it run faster.

In [9]:
import numpy as np
from timeit import timeit # guess what ... this times things.

# a simple function that creates a list of 100,000 Boolean variables - True or False
np.random.seed(444)
x = np.random.choice([False, True], size=100000)

# next a simple function that will compare each value with the previous one, and check if it is the same
# e.g. if we get "True" and then "True" this is a match (and is added to the count), but "False"/"True"
# or "True/False" is not counted
def count_transitions(x) -> int:
	count = 0
	for i, j in zip(x[:-1], x[1:]):
		if j and not i:
			count += 1
	return count # return the final count

# run the function and get the result
print(f"T1 = {count_transitions(x)}")

# here we used the vectorised numpy function that does the same thing ...
# count_nonzero. Does it produce the same result?
print(f"T2 = {np.count_nonzero(x[:-1] < x[1:])}")

T1 = 24984
T2 = 24984


We get the same result! But which is faster?

In [10]:
# pass the function calls to timeit
setup = 'from __main__ import count_transitions, x; import numpy as np; 	from timeit import timeit'

num = 1000 # how many times we run the function to get accurate estimates of time

t1 = timeit('count_transitions(x)', setup=setup, number=num)
t2 = timeit('np.count_nonzero(x[:-1] < x[1:])', setup=setup, number=num)

print('T1 time = ' + str(t1))
print('T2 time = ' + str(t2))

T1 time = 10.131675481999991
T2 time = 0.020542118999998138


We can see that the _numpy_ vectorised version is much, much faster. Let's try another - a _numba_ just-in-time compilation:

In [12]:
from numba import jit

# the @jit decorator indicates we want to compile the particular function to make it run faster
@jit
def numba_counter(x):
	np.count_nonzero(x[:-1] < x[1:])

np.random.seed(444)
x = np.random.choice([False, True], size=1000000)

setup = 'from __main__ import count_transitions, numba_counter, x; import numpy as np; from timeit import timeit; from numba import jit'

num = 1000

t1 = timeit('count_transitions(x)', setup=setup, number=num)
t2 = timeit('np.count_nonzero(x[:-1] < x[1:])', setup=setup, number=num)
t3 = timeit('numba_counter(x)', setup=setup, number=num)

print('T1 time = ' + str(t1))
print('T2 time = ' + str(t2))
print('T3 time = ' + str(t3))

<ipython-input-12-283f5f0ede2b>:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def numba_counter(x):


T1 time = 98.45009520199997
T2 time = 0.19879276200003915
T3 time = 0.6299483899999814


_numpy_ vectorisation remains the fastest. However, _numba_ just-in-time is not far behind and much faster than the basic version. I.e. if we can use a _numpy_ built-in vectorisation this is best, but if we have a unique function we can always use _numba_. 😙